## Feature Extraction Notebook

---

This notebook is used to extract features from the data. The audio is divided into windows of 1 second. The features extracted from each audio window are:

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient

#### Sections:

- [Feature Extraction](#Feature-Extraction)
  - [Specific Features Extraction](#Specific-Features-Extraction)
  - [All Features Extraction](#All-Features-Extraction)

#### Findings:

- We have highly unbalanced classes in the dataset.


In [11]:
# import all the functions
import utils as utils
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from tqdm.notebook import tqdm
import os
import importlib
import torchaudio
import torch

# import the utils module
importlib.reload(utils)

<module 'utils' from '/Users/andreaalberti/Desktop/Public_Projects/advanced-biomedical-project/notebooks/utils.py'>

In [12]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

# -------- tqdm DARK THEME --------

### Features Extraction


In [22]:
# hyperparameters
#sr = 'mix'
sr = 4000
INTERVAL = 3

# set the paths to data
BASE_DIR = "../dataset/"
ARTIFACTS_DIR = BASE_DIR + f"artifacts_{sr}/"
EXTRAHLS_DIR = BASE_DIR + f"extrahls_{sr}/"
MURMURS_DIR = BASE_DIR + f"murmurs_{sr}/"
NORMALS_DIR = BASE_DIR + f"normals_{sr}/"
EXTRASTOLES_DIR = BASE_DIR + f"extrastoles_{sr}/"

# paths to save the features
FEATURES_RAW_DIR = "../features/raw/"
# FEATURES_RAW_DIR = "../features/balanced/priori/"

PATHS = [ARTIFACTS_DIR, EXTRAHLS_DIR, MURMURS_DIR, NORMALS_DIR, EXTRASTOLES_DIR]

# REMOVE THE AUGEMENTED SAMPLES WHEN EXTRACTING BASE RAW FEATURES
# for path in PATHS:
#      remove_generated_samples(path, 'USERAUGMENTED')

### Specific Features Extraction

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user


In [14]:
def extract_features(
    dir_path: str,
    label: str,
    frame_length: float,
    sample_rate: int = 44100,
    n_mfcc: int = 13,
    melkwargs: dict = {},
    n_cqt: int = 84,
    get_mfcc: bool = True,
    get_chroma: bool = True,
    get_cqt: bool = True,
) -> list:
    """
    Extracts audio features (MFCC, chroma, RMS, spectral centroid, spectral bandwidth, spectral rolloff, zero-crossing rate)
    from audio files in the specified directory.

    Args:
        dir_path (str): The path to the directory containing audio files.
        label (str): The label associated with the extracted features.
        frame_length: The frame length for feature extraction.
        sample_rate (int, optional): The sample rate of the audio files. Defaults to 44100 Hz.
        n_mfcc (int, optional): The number of Mel-frequency cepstral coefficients (MFCCs) to extract. Defaults to 13.
        melkwargs (dict, optional): Additional arguments for Mel spectrogram computation.
        n_cqt (int, optional): The number of constant-Q transform (CQT) bins to extract. Defaults to 84.

    Returns:
        list: A list containing tensors of extracted features for each audio file.
    """
    filenames = os.listdir(dir_path)
    filenames = [file for file in filenames if not file.startswith(".")]
    frame_length = 1 / frame_length

    features = []  # List to store the features for all files

    for file in tqdm(filenames, desc=f"Extraction in progress"):
        audio, orig_sample_rate = torchaudio.load(os.path.join(dir_path, file))
        orig_sample_rate_tmp = int(orig_sample_rate / frame_length)
        audio_length = int(max(audio[0].shape) / (orig_sample_rate_tmp))

        sample_rate = orig_sample_rate

        # Reduce the audio from stereo to mono if needed
        if audio.shape[0] > 1:
            print(f"Converting stereo audio to mono for {file}...")
            audio = torch.mean(audio, dim=0).reshape(1, -1)

        features_local = []  # List to store the MFCC features for each file

        for i in range(audio_length):
            # Lazy load the audio, one sec at a time, to avoid memory issues
            audio_mono = utils.slicing(
                audio,
                offset=int(sample_rate / frame_length * i),
                num_frames=int(sample_rate / frame_length * (i + 1)),
            )

            # Get the MFCC features
            if get_mfcc:
                mfcc_features_tmp = utils.extract_mfcc(
                    audio_mono, sample_rate, n_mfcc, melkwargs
                )
            if get_chroma:
                chroma_stft = utils.extract_chroma_stft(audio_mono, sample_rate)
            if get_cqt:
                cqt = utils.extract_cqt(
                    audio_mono, sample_rate=sample_rate, n_cqt=n_cqt
                )

            #   rms = extract_rms(audio_mono)
            #   spec_cent = extract_spectral_centroid(audio_mono, sample_rate)
            #   spec_bw = extract_spectral_bandwidth(audio_mono, sample_rate)
            #   rolloff = extract_spectral_rolloff(audio_mono, sample_rate)
            #   zcr = extract_zero_crossing_rate(audio_mono)

            # Concatenate the features
            if get_mfcc:
                features_tmp = mfcc_features_tmp
            if get_chroma:
                features_tmp = chroma_stft
            if get_cqt:
                features_tmp = cqt

            if get_mfcc and get_chroma:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, chroma_stft),
                    dim=0,
                )
            if get_mfcc and get_cqt:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, cqt),
                    dim=0,
                )
            if get_chroma and get_cqt:
                features_tmp = torch.cat(
                    (chroma_stft, cqt),
                    dim=0,
                )
            if get_mfcc and get_chroma and get_cqt:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, chroma_stft, cqt),
                    dim=0,
                )
            if not get_mfcc and not get_chroma and not get_cqt:
                features_tmp = torch.tensor([])

            features_local.append(features_tmp)

        # If features_local is empty, skip the file
        if features_local == []:
            print(f"No features extracted for {file}. Skipping...")
            continue

        # Stack the MFCC features into a single tensor
        features_local = torch.stack(features_local, dim=0)

        # Attach the label in the last column
        features_local = torch.cat(
            (features_local, torch.ones(features_local.shape[0], 1) * label), dim=1
        )

        # Attach the filename index in the last column
        features_local = torch.cat(
            (
                features_local,
                torch.full((features_local.shape[0], 1), filenames.index(file)),
            ),
            dim=1,
        )

        features.append(features_local)

    print("Finished processing all files.\n")
    return features

In [15]:
# ------------------------- MFCC -------------------------
n_mfcc_list = [30, 60, 90, 120]
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "mfcc"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_mfcc in n_mfcc_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_mfcc}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                frame_length=interval,
                n_mfcc=n_mfcc,
                get_mfcc=True,
                get_chroma=False,
                get_cqt=False,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/92 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_49.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_47.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/19 [00:00<?, ?it/s]

No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrahls_4000/ features tensor is: (37, 32)
Extracting features from ../dataset/murmurs_4000/


...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
No features extracted for murmur__171_1307971016233_D.wav. Skipping...
No features extracted for murmur__193_1308078104592_C1.wav. Skipping...
No features extracted for murmur_noisymurmur_292_1311185449649_D.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
No features extracted for murmur__171_1307971016233_D1.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_F.wav. Skipping...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
No featur

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__208_1308159994503_C.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__215_1308245664733_C.wav. Skipping...
No features extracted for normal_noisynormal_189_1308075231945_A1.wav. Skipping...
No features extracted for normal__129_1306344506305_B1.wav. Skipping...
No features extracted for normal__175_1307987962616_B.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__230_1308595300880_C.wav. Skipping...
No features extracted for normal_noisynormal_163_1307104470471_D1.wav. Skipping...
No features extracted for normal__174_13079877371

Extraction in progress:   0%|          | 0/46 [00:00<?, ?it/s]

No features extracted for extrastole__216_1308245839516_B.wav. Skipping...
No features extracted for extrastole__253_1309350256198_B.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C1.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for extrastole__134_1306428161797_C1.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for extrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__215_1308245664733_B1.wav. Skipping...
No features extracted for extrastole__154_1306935608852_D2.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (64, 122)
Features extracted and saved


In [16]:
# ------------------------- CHROMA -------------------------
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
n_features = 12
type_ = "chroma"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_mfcc in n_mfcc_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_features}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                frame_length=interval,
                n_mfcc=n_mfcc,
                get_mfcc=False,
                get_chroma=True,
                get_cqt=False,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

Extracting features from ../dataset/artifacts_4000/


Extraction in progress:   0%|          | 0/92 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_49.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_47.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/19 [00:00<?, ?it/s]

No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrahls_4000/ features tensor is: (37, 14)
Extracting features from ../dataset/murmurs_4000/


...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
No features extracted for murmur__171_1307971016233_D.wav. Skipping...
No features extracted for murmur__193_1308078104592_C1.wav. Skipping...
No features extracted for murmur_noisymurmur_292_1311185449649_D.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
No features extracted for murmur__171_1307971016233_D1.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_F.wav. Skipping...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
No featur

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__208_1308159994503_C.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__215_1308245664733_C.wav. Skipping...
No features extracted for normal_noisynormal_189_1308075231945_A1.wav. Skipping...
No features extracted for normal__129_1306344506305_B1.wav. Skipping...
No features extracted for normal__175_1307987962616_B.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__230_1308595300880_C.wav. Skipping...
No features extracted for normal_noisynormal_163_1307104470471_D1.wav. Skipping...
No features extracted for normal__174_13079877371

Extraction in progress:   0%|          | 0/46 [00:00<?, ?it/s]

No features extracted for extrastole__216_1308245839516_B.wav. Skipping...
No features extracted for extrastole__253_1309350256198_B.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C1.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for extrastole__134_1306428161797_C1.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for extrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__215_1308245664733_B1.wav. Skipping...
No features extracted for extrastole__154_1306935608852_D2.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles_4000/ features tensor is: (64, 14)
Features extracted and saved
The features have already been extracted
The features have already been extracted
The features have already been extracted


In [23]:
# ------------------------- CQT -------------------------
n_cqt_list = [
    20,
    30,
    40,
    60,
    70,
]
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = INTERVAL
type_ = "cqt"  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_cqt in n_cqt_list:
    # Save the features to a file
    storing_name = f"full_data_{interval}s_{sr}hz_{n_cqt}{type_}"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE

    if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
        print("The features have already been extracted")

    else:
        for i, PATH_ in enumerate(PATHS):

            print(f"Extracting features from {PATH_}")
            features = extract_features(
                PATH_,
                label=i,
                frame_length=interval,
                n_cqt=n_cqt,
                get_mfcc=False,
                get_chroma=False,
                get_cqt=True,
            )

            # Stack the features into a single tensor
            features = torch.cat(features, dim=0).numpy()
            print(f"The shape of the {PATH_} features tensor is: {features.shape}")

            X = features[:, :-2]
            y = features[:, -2]
            filename = features[:, -1]

            local_dict = {"X": X, "y": y, "filename": filename}

            features_dict[names[i]] = local_dict

        # Save the features to a file
        np.save(FEATURES_RAW_DIR + storing_name, features_dict)

        print("Features extracted and saved")

The features have already been extracted
The features have already been extracted
The features have already been extracted
The features have already been extracted
Extracting features from ../dataset/artifacts/


Extraction in progress:   0%|          | 0/92 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_49.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_47.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/19 [00:00<?, ?it/s]

No features extracted for extrahls__201104140118.wav. Skipping...
No features extracted for extrahls__201104270459.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrahls/ features tensor is: (37, 72)
Extracting features from ../dataset/murmurs/


...

No features extracted for murmur__293_1311680805936_B1.wav. Skipping...
No features extracted for murmur__201104021355.wav. Skipping...
No features extracted for murmur__171_1307971016233_D.wav. Skipping...
No features extracted for murmur__193_1308078104592_C1.wav. Skipping...
No features extracted for murmur_noisymurmur_292_1311185449649_D.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
No features extracted for murmur__171_1307971016233_D1.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_F.wav. Skipping...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
No features extracted for murmur_noisymurmur_171_1307971016233_D.wav. Skipping...
No featur

Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__232_1308748524018_D1.wav. Skipping...
No features extracted for normal__210_1308162935880_D2.wav. Skipping...
No features extracted for normal__208_1308159994503_C.wav. Skipping...
No features extracted for normal__210_1308162935880_D1.wav. Skipping...
No features extracted for normal__215_1308245664733_C.wav. Skipping...
No features extracted for normal_noisynormal_189_1308075231945_A1.wav. Skipping...
No features extracted for normal__129_1306344506305_B1.wav. Skipping...
No features extracted for normal__175_1307987962616_B.wav. Skipping...
No features extracted for normal__143_1306763822290_C.wav. Skipping...
No features extracted for normal__147_1306523973811_C.wav. Skipping...
No features extracted for normal__238_1309194586293_B.wav. Skipping...
No features extracted for normal__230_1308595300880_C.wav. Skipping...
No features extracted for normal_noisynormal_163_1307104470471_D1.wav. Skipping...
No features extracted for normal__174_13079877371

Extraction in progress:   0%|          | 0/46 [00:00<?, ?it/s]

No features extracted for extrastole__216_1308245839516_B.wav. Skipping...
No features extracted for extrastole__253_1309350256198_B.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C1.wav. Skipping...
No features extracted for extrastole__202_1308145175747_C2.wav. Skipping...
No features extracted for extrastole__134_1306428161797_C1.wav. Skipping...
No features extracted for extrastole__190_1308076920011_C.wav. Skipping...
No features extracted for extrastole__237_1308750231222_A.wav. Skipping...
No features extracted for extrastole__215_1308245664733_B1.wav. Skipping...
No features extracted for extrastole__154_1306935608852_D2.wav. Skipping...
Finished processing all files.

The shape of the ../dataset/extrastoles/ features tensor is: (64, 72)
Features extracted and saved


### All Features Extraction

- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient


In [18]:
# FEATURES EXTRACTION
names = ["artifacts", "extrahls", "murmurs", "normals", "extrastoles"]
features_dict = {}
interval = 1
n_mfcc = 30  # 30, 120
n_cqt = 70  # 30, 70

# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

# Save the features to a file
storing_name = f"full_data_{interval}s_{sr}hz_{n_mfcc}mfcc_{n_cqt}cqt_12chroma"  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE


if os.path.exists(FEATURES_RAW_DIR + storing_name + ".npy"):
    print("The features have already been extracted")

else:
    for i, PATH_ in enumerate(PATHS):

        print(f"Extracting features from {PATH_}")
        features = utils.extract_features(
            PATH_, label=i, frame_length=interval, n_mfcc=n_mfcc, n_cqt=n_cqt
        )

        # Stack the features into a single tensor
        features = torch.cat(features, dim=0).numpy()
        print(f"The shape of the {PATH_} features tensor is: {features.shape}")

        X = features[:, :-2]
        y = features[:, -2]
        filename = features[:, -1]

        local_dict = {"X": X, "y": y, "filename": filename}

        features_dict[names[i]] = local_dict

    # Save the features to a file
    np.save(FEATURES_RAW_DIR + storing_name, features_dict)

    print("Features extracted and saved")

The features have already been extracted


In [19]:
# load the features and check the shape
features_dict = np.load(
    FEATURES_RAW_DIR + storing_name + ".npy", allow_pickle=True
).item()
for key in features_dict.keys():
    print(f'The shape of the {key} features tensor is: {features_dict[key]["X"].shape}')
    print(f'The shape of the {key} labels tensor is: {features_dict[key]["y"].shape}')
    print(
        f'The shape of the {key} filenames tensor is: {features_dict[key]["filename"].shape}'
    )
    print("-----------------------------------------------")

The shape of the artifacts features tensor is: (2001, 117)
The shape of the artifacts labels tensor is: (2001,)
The shape of the artifacts filenames tensor is: (2001,)
-----------------------------------------------
The shape of the extrahls features tensor is: (124, 117)
The shape of the extrahls labels tensor is: (124,)
The shape of the extrahls filenames tensor is: (124,)
-----------------------------------------------
The shape of the murmurs features tensor is: (1149, 117)
The shape of the murmurs labels tensor is: (1149,)
The shape of the murmurs filenames tensor is: (1149,)
-----------------------------------------------
The shape of the normals features tensor is: (2161, 117)
The shape of the normals labels tensor is: (2161,)
The shape of the normals filenames tensor is: (2161,)
-----------------------------------------------
The shape of the extrastoles features tensor is: (247, 117)
The shape of the extrastoles labels tensor is: (247,)
The shape of the extrastoles filenames t

In [20]:
s = 0
for key in features_dict.keys():
    s += features_dict[key]["X"].shape[0]
s

5682